In [18]:
import tensorflow as tf
import numpy as np

In [22]:
class naiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
    
        w_shape = (input_size, output_size)
        w_initial_values = tf.random.uniform(w_shape, minval = 0, maxval = 1e-1)
        self.w = tf.Variable(w_shape, dtype = 'float32')
        
        b_shape = (output_size, )
        b_initial_values = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_values)
        
    def __call__(self, inputs):
        self.activation(tf.matmul(inputs, self.w), self.b)
    
    @property
    def weights(self):
        return [self.w, self.b]

In [23]:
class naiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [24]:
model = naiveSequential([naiveDense(input_size = 28*28, output_size = 512, activation = tf.nn.relu),
                        naiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)])

assert len(model.weights) == 4

In [25]:
import math

In [26]:
class BatchGenerator:
    def __init__(self, images, batch_size = 120):
        assert len(images) == len(labels)
        self.index = 0
        self.images= images
        self.labels= labels
        self.batch_size = batch_size
        self.num_batches= num_batches
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.images[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [27]:
def oneTrainingStep(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(image_batch)
        per_sample_loss = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_loss)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [28]:
learning_rate = 1e-3

In [29]:
from tensorflow.keras import optimizers

In [30]:
optimizer = optimizers.SGD(learning_rate = 1e-3)

In [31]:
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))